In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import cv2
import os 
import numpy as np
import matplotlib.pyplot as plt
import math
from PIL import Image
import random
import matplotlib.image as mpimg
from imutils import paths
import sys
import operator

In [ ]:
!pip install rich
!pip install hydra-core
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 4.3 MB/s 
     |████████████████████████████████| 51 kB 7.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 151 kB 4.7 MB/s 
     |████████████████████████████████| 79 kB 2.8 MB/s 
     |████████████████████████████████| 117 kB 51.1 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=ed35a5e188fb54fde817e93d4e7f1c4e3aef6a7d8619147e57c01cbb4e3fad85
  Stored in directory: /root/.cache/pip/wheels/8b/8d/53/2af8772d9aec614e3fc65e53d4a993ad73c61daa8bbd85a873
Successfully built antlr4-python3-runtime


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 705 kB 4.3 MB/s 
     |████████████████████████████████| 419 kB 46.9 MB/s 
     |████████████████████████████████| 5.9 MB 39.4 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.


In [ ]:
sys.path.insert(0, "/content/drive/MyDrive/Github_Repos/historia_ml_handwritten_text_line_segmentation")
sys.path.insert(1, "/content/drive/MyDrive/Github_Repos/historia_ml_handwritten_text_line_segmentation/src/utils")

In [ ]:
import pytorch_lightning as pl
from src.utils.display_image_from_path import img_show
from src.utils.display_image_from_array import img_show_from_arr

In [ ]:
img_path = "/content/drive/MyDrive/Images/OMERFAZILAKOZ17-211.jpeg"
img = cv2.imread(img_path)
img_mask = cv2.imread("/content/drive/MyDrive/output/test_page_predicted_masks/OMERFAZILAKOZ17-211_mask_480x480.png")

In [ ]:
img_show_from_arr(img)
img_show_from_arr(img_mask)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
cfg = {
    "img_path": img_path,
    "x_ratio": 0.05,
    "y_ratio": 0.1,
    "row_height_limit_ratio": 0.7,
    "line_gap_ratio": 0.08,
    "safety_gap_x": 10,
    "safety_gap_y": 6
}

In [ ]:
def delete_border_lines(mask, x_ratio, y_ratio):
  height, width, depth = mask.shape
  vertical_limit = int(height * y_ratio)
  horizontal_limit = int(width * x_ratio)
  # delete upper border
  for i in range(0, vertical_limit):
    for j in range(width):
      mask[i][j][0] = 0
      mask[i][j][1] = 0
      mask[i][j][2] = 0
  # delete left border
  for i in range(height):
    for j in range(0, horizontal_limit):
      mask[i][j][0] = 0
      mask[i][j][1] = 0
      mask[i][j][2] = 0
  # delete lower border
  for i in range(height-vertical_limit, height):
    for j in range(width):
      mask[i][j][0] = 0
      mask[i][j][1] = 0
      mask[i][j][2] = 0
  # delete right border
  for i in range(height):
    for j in range(width - horizontal_limit, width):
      mask[i][j][0] = 0
      mask[i][j][1] = 0
      mask[i][j][2] = 0
  return mask


def next_neighbor_as_zero(height, width, mask):
  pixels_having_0_as_neighbor = []
  for i in range(1, height-1):
    for j in range(1, width-1):
      if mask[i][j][0] != 0:
        if mask[i-1][j][0] == 0: # downstairs neighbor
          pixels_having_0_as_neighbor.append((i, j))
        elif mask[i][j-1][0] == 0: # left neighbor
          pixels_having_0_as_neighbor.append((i, j))
        elif mask[i+1][j][0] == 0: # upstairs neighbor
          pixels_having_0_as_neighbor.append((i, j))
        elif mask[i][j+1][0] == 0: # right neighbor
          pixels_having_0_as_neighbor.append((i, j))
  return pixels_having_0_as_neighbor


def get_row_heights(bbox_y_pairs):
  row_heights = []
  for pair in bbox_y_pairs:
    row_height = abs(pair[0]-pair[1])
    row_heights.append(row_height)
  return row_heights


def find_image_page_type(cols, line_gap_ratio):
  min_col_num = min(cols)
  max_col_num = max(cols)
  lines_area_width = abs(min_col_num - max_col_num) 
  line_gap_threshold = lines_area_width * line_gap_ratio
  for i in range(len(cols)-1):
    if (cols[i+1] - cols[i]) > line_gap_threshold:
      return "two page image"
  return "one_page_image"


def create_row_patches(y1, y2, coords):
  row_patch_coords = []
  for coord in coords:
    if coord[0] >= y1 and coord[0] <= y2:
      row_patch_coords.append(coord)
  return row_patch_coords

def create_unique_patch_cols(patch):
  patch_cols = [patch[i][1] for i in range(len(patch))]
  patch_cols = sorted(patch_cols)
  patch_cols = np.array(patch_cols)
  patch_unique_cols = np.unique(patch_cols)
  return patch_unique_cols

def create_bbox_list(rows, cols, line_gap_ratio, coords):
  page_type = find_image_page_type(cols, line_gap_ratio)
  bbox_list = []
  if page_type == "two page image":
    line_gap_threshold = abs(min(cols) - max(cols)) * line_gap_ratio
    for row_pair in rows:
      y1 = row_pair[0]
      y2 = row_pair[1]
      patch = create_row_patches(row_pair[0], row_pair[1], coords)
      patch_unique_cols = create_unique_patch_cols(patch)
      x_min = min(patch_unique_cols)
      x_max = max(patch_unique_cols)
      for i in range(len(cols)-1):
        if cols[i] >= x_min and cols[i] <= x_max:
          if abs(cols[i] - cols[i+1]) > line_gap_threshold:
            bbox_list.append((x_min, y1, cols[i], y2))
            bbox_list.append((cols[i+1], y1, x_max, y2))
  else:
    for row_pair in rows:
      y1 = row_pair[0]
      y2 = row_pair[1]
      patch = create_row_patches(row_pair[0], row_pair[1], coords)
      patch_unique_cols = create_unique_patch_cols(patch)
      x_min = min(patch_unique_cols)
      x_max = max(patch_unique_cols)
      bbox_list.append((x_min, y1, x_max, y2))
  return bbox_list


def create_line_image_from_bbox(bbox, img, safety_gap_x, safety_gap_y):
  x1, y1, x2, y2 = bbox
  row = img[y1-safety_gap_y:y2+safety_gap_y, x1-safety_gap_x:x2+safety_gap_x, :]
  return row
  

def create_row_images(bbox_list, img, safety_gap_x, safety_gap_y):
  rows = []
  for i in range(len(bbox_list)):
    row = create_line_image_from_bbox(bbox_list[i], img, safety_gap_x, safety_gap_y)
    rows.append(row)
  return rows


In [ ]:
def get_bboxes_from_mask(mask, x_ratio, y_ratio, row_height_limit_ratio, line_gap_ratio):
  next_neighbor0_rows = []
  bbox_ycoords = []
  row_patches = []
  next_neighbor0_cols = []
  mask = img_mask.copy()
  mask = delete_border_lines(mask, x_ratio, y_ratio)

  height, width, depth = img.shape
  next_neighbor0_coords = next_neighbor_as_zero(height, width, mask)
  
  for coord in next_neighbor0_coords:
    next_neighbor0_rows.append(coord[0])
  next_neighbor0_rows = np.array(next_neighbor0_rows)
  next_neighbor0_unique_rows = np.unique(next_neighbor0_rows)

  for coord in next_neighbor0_coords:
    next_neighbor0_cols.append(coord[1])
  next_neighbor0_cols = np.array(next_neighbor0_cols)
  next_neighbor0_unique_cols = np.unique(next_neighbor0_cols)

  bbox_ycoords.append(next_neighbor0_unique_rows[0])
  for i in range(len(next_neighbor0_unique_rows)-1):
    if abs(next_neighbor0_unique_rows[i] - next_neighbor0_unique_rows[i+1]) > 3:
      bbox_ycoords.append(next_neighbor0_unique_rows[i])
      bbox_ycoords.append(next_neighbor0_unique_rows[i+1])
  bbox_ycoords.append(next_neighbor0_unique_rows[-1])
  bbox_ycoords_pair = [(bbox_ycoords[i], bbox_ycoords[i+1]) for i in range(0, len(bbox_ycoords), 2)]

  row_heights = get_row_heights(bbox_ycoords_pair)
  mean_row_height = np.mean(row_heights)
  row_height_limit = mean_row_height * row_height_limit_ratio
  for pair in bbox_ycoords_pair:
    if abs(pair[0] - pair[1]) > row_height_limit:
      row_patches.append(pair)

  bboxes = create_bbox_list(row_patches, next_neighbor0_unique_cols, line_gap_ratio, next_neighbor0_coords)
  return bboxes

In [ ]:
bboxes = get_bboxes_from_mask(img_mask, cfg["x_ratio"], cfg["y_ratio"], cfg["row_height_limit_ratio"], cfg["line_gap_ratio"])
bboxes

In [ ]:
img_path.split("/")[-1].split(".")[0]

'OMERFAZILAKOZ17-211'

In [ ]:
x = img_path.split("/")[:-1]
path = ""
for i in range(1, len(x)):
  path += x[i] + "/"

path 

'content/drive/MyDrive/Images/'

In [ ]:
rows = create_row_images(bboxes, img, cfg["safety_gap_x"], cfg["safety_gap_y"])
img_show_from_arr(img)
for row in rows:
  img_show_from_arr(row)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def save_line_images(lines, img_path, rows):
    filename = img_path.split("/")[-1].split(".")[0]
    folder_path = ""
    img_path_split = img_path.split("/")[:-1]
    for i in range(1, len(img_path_split)):
      folder_path += img_path_split[i] + "/"

    if not (os.path.exists(folder_path + filename + " Lines")):
        os.mkdir(filename + " Lines")
        
    for i in range(len(rows)):
      cv2.imwrite(folder_path + filename + "_line_" + i + ".png", lines[i])